In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [ ]:
data_yaml = "/kaggle/input/vehicle-detection-yolov8-dataset/data.yaml"

In [ ]:
models = ["yolov8n.pt", "yolov8m.pt", "yolov8x.pt"]

In [ ]:
# Training, Validation & Testing
results = {}
for model_name in models:
    print(f"Training {model_name}...")
    
    # Load YOLO model
    model = YOLO(model_name).to(device)
    
    # Train the model
    model.train(data=data_yaml, epochs=30, imgsz=640, batch=8, device=device)
    
    # Validate the model
    metrics = model.val()
    results[model_name] = metrics
    
    # Test the model
    test_results = model.val(split='test')
    
    print(f"Results for {model_name}:")
    print(f"mAP50: {metrics.map50()}")
    print(f"mAP50-95: {metrics.map()}")
    print(f"Precision: {metrics.mp()}")
    print(f"Recall: {metrics.mr()}")
    
    # Save results for later comparison
    results[model_name] = {
        "mAP50": metrics.map50(),
        "mAP50-95": metrics.map(),
        "Precision": metrics.mp(),
        "Recall": metrics.mr()
    }

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Training, Validation & Testing
results = {}
for model_name in models:
    print(f"Training {model_name}...")
    
    # Load YOLO model
    model = YOLO(model_name).to(device)
    
    # Train the model
    model.train(data=data_yaml, epochs=30, imgsz=640, batch=8, device=device)
    
    # Validate the model
    metrics = model.val()
    results[model_name] = metrics
    
    # Test the model
    test_results = model.val(split='test')
    
    
    print(f"Results for {model_name}:")
    print(f"mAP50: {metrics.box.map50}")
    print(f"mAP50-95: {metrics.box.map}")
    print(f"Precision: {metrics.box.p.mean()}")
    print(f"Recall: {metrics.box.r.mean()}")
    
    # Save results for later comparison
    results[model_name] = metrics

# Plot Precision, Recall, mAP
plt.figure(figsize=(10, 5))
metrics_map = {
      "mAP50": lambda r: r.box.map50,
    "mAP50-95": lambda r: r.box.map,
    "Precision": lambda r: r.box.p.mean(),
    "Recall": lambda r: r.box.r.mean(),
}

for metric, func in metrics_map.items():
    values = [func(results[m]) for m in models]  # Call method on results[m]
    plt.plot(models, values, marker='o', label=metric)

plt.legend()
plt.xlabel("Model")
plt.ylabel("Score")
plt.title("Performance Comparison")
plt.grid()
plt.show()




In [ ]:
# Confusion Matrix
for model_name in models:
    print(f"Confusion Matrix for {model_name}:")
    model = YOLO(model_name).to(device)
    metrics = model.val()
    conf_matrix = metrics.confusion_matrix
    
    if conf_matrix is not None and hasattr(conf_matrix, "matrix") and conf_matrix.matrix is not None:
        sns.heatmap(conf_matrix.matrix, annot=True, fmt='d', cmap='Blues')
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix - {model_name}")
        plt.show()
    else:
        print(f"Confusion Matrix is empty for {model_name}. Skipping heatmap.")

print("Training and evaluation completed!")

